# Talktorial 11 (part B)

# Structure-based CADD using online APIs/servers

__Developed at AG Volkamer, Charité__

Dr. Jaime Rodríguez-Guerra, Dominique Sydow

## Aim of this talktorial

> This is part B of the "Online webservices" talktorial:
>
> - 11a. Querying KLIFS & PubChem for potential kinase inhibitors
> - __11b. Docking the candidates against the target obtained in 11a__
> - 11c. Assessing the results and comparing against known data


After obtaining input structures we will use molecular docking software to find good protein-ligand poses.

## Learning goals

### Theory

- Molecular docking basics
- Available software
- Known limitations

### Practical

- Prepare the structures
- Guess the binding site
- Run the docking calculation
- Save the results

## References

* Chapter on Structure‐Based Virtual Screening, [Ch. 6.8 in "Applied Chemoinformatics: Achievements and Future Opportunities" (2018)](https://onlinelibrary.wiley.com/doi/book/10.1002/9783527806539)
* How to benchmark docking software ([_J. Med. Chem._ (2006), __49__, 23, 6789–6801](https://pubs.acs.org/doi/abs/10.1021/jm0608356))
* A review on molecular docking software ([_Biophysical Reviews_ (2017), __9__, 2, 91–102](https://www.ncbi.nlm.nih.gov/pubmed/28510083))
* DoGSiteScorer, a program to identify binding sites.
   * [_J. Chem. Inf. Model._ (2010), __50__, 11, 2041-2052](https://doi.org/10.1021/ci100241y)
   * [_Bioinformatics_ (2012), __28__, 15, 2074–2075](https://doi.org/10.1093/bioinformatics/bts310)
* SwissDock, a docking program.
   * [_Nucleic Acids Res._ (2011), __39__, W270-7.](https://academic.oup.com/nar/article/39/suppl_2/W270/2506492)
   * [_J Comput Chem._ (2011), __32__, 10, 2149-59](https://onlinelibrary.wiley.com/doi/abs/10.1002/jcc.21797)
* OPAL webservices, which can run software remotely on demand
   * Manuscript: [_Nucleic Acids Res._ (2010), __38__, W724-31](https://academic.oup.com/nar/article/38/suppl_2/W724/1122840)
   * Documentation: [Opal: Simple Web Services Wrappers for Scientific Applications](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.533.7960&rep=rep1&type=pdf)
* AutoDock Vina, a docking program ([_J Comput Chem._ (2010), __31__, 455–461](https://onlinelibrary.wiley.com/doi/abs/10.1002/jcc.21334))

***

## Theory

### Molecular docking

Adapted from [Pagadala, Syed & Tuszynski, Biophysical Reviews 2017, 9, 2, 91–102](https://link.springer.com/article/10.1007%2Fs12551-016-0247-1):

> Molecular docking methodology explores the behavior of small molecules in the binding site of a target protein. The applied software performs a search algorithm in which the conformation of the ligand is evaluated recursively until the convergence to the minimum energy is reached. Finally, an affinity scoring function, ΔG (U total in kcal/mol), is employed to rank the candidate poses as the sum of the electrostatic and van der Waals energies. The driving forces for these specific interactions in biological systems aim toward complementarities between the shape and electrostatics of the binding site surfaces and the ligand or substrate.

These scoring functions are tailored to be fast to compute, and thus their accuracy is often poorer than in other molecular modeling methods. Normally, they are based on:

- Molecular mechanics principles
- Knowledge-based potentials
- Shape and geometric complementarity

To reduce the search space dimensionality, some approximations are often applied:

- The protein structure is considered mostly rigid, with maybe some side chains in the vicinities of the search area allowed to explore a limited set of conformations (rotamers).
- The ligand can be considered rigid in virtual screening studies, but it is often allowed to freely explore its bond torsions in more detailed calculations. A compromise between both options is defining a set of probable conformations beforehand.

#### Examples of existing software

- Commercial
    - GOLD
    - Schrödinger
    - FlexX
- Free (or free for academics)
    - AutoDock
    - AutoDock Vina
    - DOCK
    - OpenEye

### Known limitations

The following approximations could introduce artifacts in the calculation. 

- Since the protein is mostly rigid, the dynamic, adaptive nature of the protein-ligand binding is barely explored. This can result in some false positives: even if the ligand finds a suitable pose in the binding pocket, this position is not guaranteed until the protein is allowed to explore near-minima conformations. In other words, short molecular dynamics trajectories are always recommended to check that the ligand stays in the binding pocket.
- The scoring function must be cheap to resolve. While the accuracy is good enough to distinguish good poses from bad poses, it can have problems sorting the best poses. For example, while most popular docking programs are able to find the experimental pose in their calculations, this pose is rarely the best one of the proposed set.
- To reduce the computational cost, docking procedures are only performed in a subset of the protein (normally around a known binding pocket). Choosing the correct binding pocket becomes then another question in the CADD pipeline. 
- To leverage the accuracy of the calculation, the structures must be reduced accordingly. Protonation states of amino acids and the ligands can be tricky to get right, especially in the case of (potential) tautomers. This introduces yet another cause to obtain wrong results.

Despite all these limitations, docking calculations are still popularly used in all pharma laboratories, along with other types of molecular simulation. In this part of the talktorial we will learn how to:

1. Prepare the protein and ligands obtained in part A  (locally)
2. Estimate the most probable binding pocket (online)
3. Run the docking calculations (online)

***

### Preparation of structures

We will use the AutoDock Vina installation present in the OPAL webservices. However, this requires preparing the structures beforehand. The recommended approach is using the preparation scripts present in [AutoDockTools](http://autodock.scripps.edu/resources/adt). Although these tools are distributed on their own, and are only compatible with Python 2, we have prepared a [Python 3-ready fork](https://github.com/jaimergp/autodocktools-prepare-py3k) containing the subset needed for protein and ligand preparations. This should be enough for our needs.


### Binding pocket prediction

The docking calculation will work best if we perform it in a reasonably small search space, normally covering a single binding 
pocket. To guesstimate the best one, we can use DoGSiteScorer, available for free and online at Proteins.plus.

#### Proteins.plus DoGSiteScorer

* Role: Interactive web interface for several CADD tools
* Website: http://proteins.plus
* API: Yes, REST-based. Simple enough to apply bare `requests`
* Documentation: https://proteins.plus/help/index
* Literature:
   * [_J. Chem. Inf. Model._ (2010), __50__, 11, 2041-2052](https://doi.org/10.1021/ci100241y)
   * [_Bioinformatics_ (2012), __28__, 15, 2074–2075](https://doi.org/10.1093/bioinformatics/bts310)

> Automated prediction of protein active sites is essential for large-scale protein function prediction, classification, and druggability estimates. In this work, we present DoGSite, a new structure-based method to predict active sites in proteins based on a Difference of Gaussian (DoG) approach which originates from image processing. In contrast to existing methods, DoGSite splits predicted pockets into subpockets, revealing a refined description of the topology of active sites. DoGSite correctly predicts binding pockets for over 92% of the PDBBind and the scPDB data set, being in line with the best-performing methods available. In 63% of the PDBBind data set the detected pockets can be subdivided into smaller subpockets. The cocrystallized ligand is contained in exactly one subpocket in 87% of the predictions. Furthermore, we introduce a more precise prediction performance measure by taking the pairwise ligand and pocket coverage into account. In 90% of the cases DoGSite predicts a pocket that contains at least half of the ligand. In 70% of the cases additionally more than a quarter of the respective pocket itself is covered by the cocrystallized ligand. Consideration of subpockets produces an increase in coverage yielding a success rate of 83% for the latter measure.

### Docking

There are a couple of webservices available online for free use: SwissDock and OPAL webservices (which includes AutoDock Vina).

#### SwissDock

* Role: Perform docking calculations
* Website: http://www.swissdock.ch
* API: Yes, SOAP-based. No official client, use `suds-community`.
* Documentation: http://www.swissdock.ch/pages/soap_access
* Literature:
   * [_Nucleic Acids Res._ (2011), __39__, W270-7.](https://academic.oup.com/nar/article/39/suppl_2/W270/2506492)
   * [_J Comput Chem._ (2011), __32__, 10, 2149-59](https://onlinelibrary.wiley.com/doi/abs/10.1002/jcc.21797)

> SwissDock, a web service to predict the molecular interactions that may occur between a target protein and a small molecule.
> SwissDock is based on the docking software EADock DSS, whose algorithm consists of the following steps:
> 1. Many binding modes are generated either in a box (local docking) or in the vicinity of all target cavities (blind docking).
> 2. Simultaneously, their CHARMM energies are estimated on a grid.
> 3. The binding modes with the most favorable energies are evaluated with FACTS, and clustered.
> 4. The most favorable clusters can be visualized online and downloaded on your computer.


#### OPAL webservices

* Role: CADD as a service
* Website: http://nbcr-222.ucsd.edu/opal2/dashboard
* API: Yes, SOAP-based. No official client, use `suds-community`.
* Documentation: http://nbcr-222.ucsd.edu/opal2/dashboard?command=docs (currently offline)
* Literature:
   * Manuscript: [_Nucleic Acids Res._ (2010), __38__, W724-31](https://academic.oup.com/nar/article/38/suppl_2/W724/1122840)
   * Documentation: [Opal: Simple Web Services Wrappers for Scientific Applications](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.533.7960&rep=rep1&type=pdf)
    
> Biomedical applications have become increasingly complex, and they often require large-scale high-performance computing resources with a large number of processors and memory. The complexity of application deployment and the advances in cluster, grid and cloud computing require new modes of support for biomedical research. Scientific Software as a Service (sSaaS) enables scalable and transparent access to biomedical applications through simple standards-based Web interfaces. Towards this end, we built a production web server (http://ws.nbcr.net) in August 2007 to support the bioinformatics application called MEME. The server has grown since to include docking analysis with AutoDock and AutoDock Vina, electrostatic calculations using PDB2PQR and APBS, and off-target analysis using SMAP. All the applications on the servers are powered by Opal, a toolkit that allows users to wrap scientific applications easily as web services without any modification to the scientific codes, by writing simple XML configuration files. Opal allows both web forms-based access and programmatic access of all our applications. The Opal toolkit currently supports SOAP-based Web service access to a number of popular applications from the National Biomedical Computation Resource (NBCR) and affiliated collaborative and service projects. In addition, Opal's programmatic access capability allows our applications to be accessed through many workflow tools, including Vision, Kepler, Nimrod/K and VisTrails. From mid-August 2007 to the end of 2009, we have successfully executed 239,814 jobs. The number of successfully executed jobs more than doubled from 205 to 411 per day between 2008 and 2009. The Opal-enabled service model is useful for a wide range of applications. It provides for interoperation with other applications with Web Service interfaces, and allows application developers to focus on the scientific tool and workflow development. Web server availability: http://ws.nbcr.net.

## Practice


### Get files from part A

First, we will define some variables pointing to the files we obtained in part A:

- `PROTEIN`: path to the `.mol2` file containing the structure of the target, without any ligands or ions
- `COMPLEX`: path to the `.pdb` file containing the structure of the target AND the native ligand in its binding site
- `SMILES_FILE`: path to the `.txt` file containing the SMILES representations for all the similar compounds found in PubChem
- `smiles`: list of the SMILES string contained in `SMILES_FILE`

In [1]:
PROTEIN = "data/protein.mol2"
COMPLEX = "data/complex.pdb"
SMILES_FILE = "data/similar_smiles.txt"
with open(SMILES_FILE) as f:
    smiles = [line.strip() for line in f]

### Use SwissDock

SwissDock uses a SOAP interface, so we will need to install `suds` for that.

> Notice: SwissDock servers are not working lately. Go to the OPAL alternative below!

In [2]:
from suds.client import Client
import zlib
import string
import requests

def swissdock_client():
    # Server seems to be down at the moment...
    # http://swissdock.vital-it.ch/soap/ replies with 503 Unavailable
    # because it points to wrong domain... patch it?
    SWISSDOCK_WSDL_URL = "http://www.swissdock.ch/soap/wsdl"
    r = requests.get("http://www.swissdock.ch/soap/wsdl")
    r.raise_for_status()
    WSDL = r.text.replace("http://swissdock.vital-it.ch/soap/", "http://www.swissdock.ch/soap/")
    with open("data/swissdock.wsdl", "w") as f:
        f.write(WSDL)
    HERE = _dh[0]
    return Client(f"file://{HERE}/data/swissdock.wsdl")


def prepare_protein(client, protein):
    """
    Given a PDB file (string contents), returns PSF and CRD
    """
    encoded_protein = zlib.compress(protein.encode('utf-8'))
    job_id = client.service.prepareTarget(target=encoded_protein)
    while True:
        result = client.service.isTargetPrepared(jobID=job_id)
        if result is None:
            raise ValueError("No such a job present")
        if result in (False, "false", 0):
            time.sleep(5)
        else:  # ready!
            break
    protein_files = client.service.getPreparedTarget(job_id)
    if protein_files is None or len(protein_files) != 2:
        raise ValueError("Could not prepare protein!")
    return protein_files
            

def prepare_ligand(client, ligand):
    """
    Given a MOL2 file (string contents), returns PDB, RTF, PAR.
    
    Ligand must be protonated beforehand!
    """
    encoded_ligand = zlib.compress(ligand.encode('utf-8'))
    job_id = client.service.prepareLigand(ligand=encoded_ligand)
    while True:
        result = client.service.isLigandPrepared(jobID=job_id)
        if result is None:
            raise ValueError("No such a job present")
        if result in (False, "false", 0):
            time.sleep(5)
        else:  # ready!
            break
    ligand_files = client.service.getPreparedLigand(job_id)
    if ligand_files is None or len(ligand_files) != 3:
        raise ValueError("Could not prepare ligand!")
    return ligand_files

def dock(client, protein, ligand, name=None):
    protein_psf, protein_crd = prepare_protein(client, protein)
    ligand_pdb, ligand_rtf, ligand_par = prepare_ligand(client, ligand)
    
    if name is None:
        name = "teachopencadd" + ''.join([random.choice(string.ascii_letters) for _ in range(5)])
    job_id = client.service.startDocking(
        protein_psf, protein_crd,
        ligand_pdb,
        [ligand_rtf],
        [ligand_par],
        name)
    if job_id in (None, "None"):
        raise ValueError("Docking job could not be submitted")
    while not client.service.isDockingTerminated(job_id):
        time.sleep(5)
    all_files = client.service.getPredictedDockingAllFiles(job_id)
    with open('docking_results.zip', 'w') as f:
        f.write(all_files)
    target, docked = client.service.getPredictedDocking(job_id)
    client.service.forget(job_id)
    return target, docked

def smiles_to_pdb(s, out='output.pdb'):
    m = Chem.AddHs(Chem.MolFromSmiles(s))
    AllChem.EmbedMolecule(m)
    if out is None:
        return Chem.MolToPDBBlock(m)
    Chem.MolToPDBFile(m, out)

In [3]:
try:
    import Mol2Writer
except ImportError:
    # Ugly hack to get Mol2 writer/readers in RDKit
    import os
    working_dir = os.getcwd()
    os.chdir(_dh[0])
    !wget https://raw.githubusercontent.com/rdkit/rdkit/60081d31f45fa8d5e8cef527589264c57dce7c65/rdkit/Chem/Mol2Writer.py > /dev/null
    os.chdir(working_dir)
    import Mol2Writer

In [4]:
def step_03_swissdock(protein_pdb, ligand_smiles):
    ligand = Chem.AddHs(Chem.MolFromSmiles(ligand_smiles))
    AllChem.EmbedMolecule(ligand)
    ligand_mol2 = Mol2Writer.MolToCommonMol2Block(ligand)
    client = swissdock_client()
    return dock(client, protein_pdb, ligand_mol2)

> _The cell below has been disabled because SwissDock servers are currently unavailable. If you insist on running it, you will need to convert the protein mol2 to PDB first. You can use OpenBabel for that (installed in part C). Feel free to define a convenience_ `mol2_to_pdb` _function._

### Perform docking with OPAL webservices
SwissDock is not working recently, so we can resort to yet another webservice. The interface is a bit more rudimentary, but it should work. However, protein and ligand must be prepared locally with `AutoDockTools`. We have prepared a Python 3-ready fork, but it's not well tested. It seems to work well enough for our purposes here, though.

You can install it with:

The protocol for the docking calculation is the following:

1. Prepare the protein and the ligands with AutoDockTools (locally)
2. Find the best possible binding pocket with [Proteins.plus' DoGSiteScorer](https://proteins.plus/2ozr#dogsite). We will use this information to configure the Vina calculation (geometric center and size of the search space)
3. Run the Vina calculation on OPAL

In [5]:
import time
import os
from io import StringIO

#### Prepare the structures

Preparing the structure simply involves running the appropriate parts of the `AutoDockTools` library:

- Reading the structure file with `MolKit`
- Applying the correct preparer: `AD4ReceptorPreparation` for the protein, `AD4LigandPreparation` for the ligand.

The preparation itself will take care of things such as:

- Adding hydrogens to the protein and ligand
- Removing strange residues that do not belong to the protein
- Assigning atom types and partial charges
- Recognizing torsionable branches in the ligand

The results are written to disk as `PDBQT` files.

In [6]:
######################
#
# Structure preparation
#
######################

import MolKit
from AutoDockTools.MoleculePreparation import AD4ReceptorPreparation, AD4LigandPreparation

def opal_prepare_protein(protein):
    """
    AutoDock expects PDBQT files
    """
    mol = MolKit.Read(protein)[0]
    mol.buildBondsByDistance()
    RPO = AD4ReceptorPreparation(mol, outputfilename=protein+'.pdbqt')
    return protein + '.pdbqt'

def opal_prepare_ligand(ligand):
    """
    AutoDock expects PDBQT files
    """
    mol = MolKit.Read(ligand)[0]
    mol.buildBondsByDistance()
    RPO = AD4LigandPreparation(mol, outputfilename=ligand+'.pdbqt')
    return ligand + '.pdbqt'

#### Guess the binding sites

The DoGSiteScorer installation present in Proteins.plus provides a REST API that is very easy to use, if you only need to process a protein present in the PDB database (see `dogsite_scorer_submit_with_pdbid`). However, since we are using structures provided by KLIFS, we cannot guarantee that the position and orientation of the structure present in the official PDB registries match the one present in KLIFS. Although we could superpose both and then apply the resulting transformation matrix to the obtained pockets, it would be easier to simply upload our PDB file to Proteins.plus, as provided in the standard web interface.

However, the REST API does not provide such an option, so we have to reverse engineer the process. To locate the appropriate HTTP requests, you would need to open the Network tab in the Chrome Developer Tools and start writing down which requests were performed as you use the website normally. Authenticity tokens and HTTP headers are key to obtain a valid request here!

The results of this research are consolidated in the `dogsite_scorer_submit_with_custom_pdb` function. If you are interested in the technical details, read through the comments in the function. 

> For this approach, you will also need `BeautifulSoup` to parse some HTML code during the binding site guessing:

With this method we can now obtain the geometric center of the most probable binding pocket, as well as its size. Both values are needed to configure the Vina calculation.

In [1]:
######################
#
# Guess binding pocket
#
######################

from bs4 import BeautifulSoup
import requests

def dogsite_scorer_submit_with_pdbid(pdbid, chain):
    """
    This is the official API, but they only allow PDB codes
    
    Parameters
    ----------
    pdbid : str
        4-letter PDB identifier
    chain : str
        Structure chain to be analyzed
    
    Returns
    -------
    str
        URL that can be queried to get updates on whether the job is running or finished
    """
    # Submit job to Proteins.plus
    r = requests.post("https://proteins.plus/api/dogsite_rest",
        json={
            "dogsite": {
                "pdbCode": pdbid,
                "analysisDetail": "1",
                "bindingSitePredictionGranularity": "1",
                "ligand": "",
                "chain": chain
            }
        },
        headers= {'Content-type': 'application/json', 'Accept': 'application/json'}
    )

    r.raise_for_status()
    # We have to query location for updates on the calculation
    return r.json()['location']

def dogsite_scorer_submit_with_custom_pdb(pdbfile):
    """
    In order to upload a custom PDB, we have to mimic the actual HTML frontend:
    
    1. Obtain the CSRF token out the HTML meta headers
    2. Post the file to upload it
    3. The returned HTML page will contain the URL ID, which in turn will allow
       us to obtain the internal shorthand job ID. We can use that one to
       retrieve the public job API ID by mimicking the async calls that the
       webserver performs in the frontend (as if we were using the web interface).
    4. Once we obtain the public job ID we can switch to using the REST API.
    """
    # We need to use a `session` to store intermediate cookies during the process
    session = requests.Session()
    r = session.get("https://proteins.plus/")
    r.raise_for_status()
    # The homepage contains the CSRF token needed to validate our request
    # Otherwise it wouldn't be safe! We have to use that throughout our requests
    # so the best way is to set it as part of the session HTTP headers
    html = BeautifulSoup(r.text)
    token = html.find('input', {'name': 'authenticity_token'}).attrs['value']
    session.headers['X-CSRF-Token'] = token

    # 1. Upload file
    with open(pdbfile, 'rb') as f:
        r = session.post("https://proteins.plus", files={'pdb_file[pathvar]': f})
    r.raise_for_status()

    # If the REST API supported file uploads, we would have the public ID already
    # but in the meantime you will have to work around it this way
        
    # 2. Get internal location id
    html = BeautifulSoup(r.text)
    pdb_id = html.find('input', {'name': "dogsite[pdbCode]"}).attrs['value']

    # 3. Get the internal job ID
    session.headers['Referer'] = "https://proteins.plus" + pdb_id
    r = session.post(f"https://proteins.plus/{pdb_id}/dogsites",
            json={"dogsite": {"pdbCode": pdb_id}},
            headers= {'Content-type': 'application/json', 
                      'Accept': 'application/json'}
    )
    r.raise_for_status()
    job_id = r.json()['job_id']
    time.sleep(3)  # wait a bit before continuing so the server can process the request
    
    # 4. Get the public job ID
    while True:
        r = session.get(f"https://proteins.plus/{pdb_id}/dogsites/{job_id}?_={round(time.time())}",
                        headers= {
                            'Accept': 'application/json, text/javascript, */*',
                            'Sec-Fetch-Mode': 'cors',
                            'Sec-Fetch-Site': 'same-origin',
                            # this line below makes all the difference, apparently
                            # otherwise, error 406 is thrown
                            'X-Requested-With': 'XMLHttpRequest'}
                       )
        r.raise_for_status()
        if 'Calculation in progress...' in r.text:  # not finished yet
            time.sleep(5)
            continue
        if 'Error during DogSiteScorer calculation' in r.text:  # malformed file?
            raise ValueError('Could not run DoGSiteScorer!')
        break
    
    results_id = None
    for lines in r.text.splitlines():
        for line in lines.split('\\n'):
            if 'results/dogsite' in line:
                results_id = line.split('/')[3]
                break
    if results_id is None:
        raise ValueError(r.text)
        
    return f"https://proteins.plus/api/dogsite_rest/{results_id}"
    

def dogsite_scorer_guess_binding_site(protein):
    """
    Use Proteins.plus' DoGSiteScorer to retrieve most probable binding site in protein.
    """
    if len(protein) == 4:  # pdb code
        job_location = dogsite_scorer_submit_with_pdbid(protein)
    elif protein.endswith('.pdb'):
        job_location = dogsite_scorer_submit_with_custom_pdb(protein)
    else:
        raise ValueError("`protein` must be a PDB ID or a path to a .pdb file!")
    
    # Check when the calculation has finished
    while True:
        result = requests.get(job_location)
        result.raise_for_status()  # if it fails, it will stop here
        if result.status_code == 202:  # still running
            time.sleep(5)
            continue
        break
    
    # The residues files contain the geometric center and radius as a comment in the PDB file
    # first file (residues[0]) is the best scored pocket
    pdb_residues = requests.get(result.json()['residues'][0]).text
    for line in pdb_residues.splitlines():
        line = line.strip()
        if line.startswith('HEADER') and 'Geometric pocket center at' in line:
            fields = line.split()
            center = [float(x) for x in fields[5:8]]
            radius = float(fields[-1])
            break
    return center, radius  # this is what we need for our Vina calculation

#### Run Vina on OPAL


Once we have (1) prepared the protein and ligand, and (2) guessed the search area, we can submit the actual calculation to the OPAL web servers. This involves:

1. Initializing the SOAP client with `suds`
2. Encoding the files as [`base64` strings](https://en.wikipedia.org/wiki/Base64#Examples) so they can be attached to the SOAP XML request
3. Submitting the job request and getting the job ID back
4. Querying the server with the job ID to check whether the calculation is finished or not
5. Downloading the relevant output files with `requests`

> These steps are not very well documented in the OPAL website (actually, the documentation is [unavailable](http://rocce-vm0.ucsd.edu/data/docs/opal/documentation.html)), so they were figured out by exploring the source code present in some of the [UCSF Chimera modules](http://plato.cgl.ucsf.edu/trac/chimera/browser/trunk/libs/WebServices/opal_client.py) that rely on these servers.

Since the calculation usually takes 5-15 minutes, for the 4th step we will use some of the Jupyter goodies and update the contents of the output file in realtime (see function `iprint()`). That way, we can check the progress instead of blindly trusting that the calculation is running ok!

In [ ]:
######################
#
# Run calculation
#
######################

from suds.client import Client
from IPython.display import display, clear_output, HTML
from rdkit import Chem
from rdkit.Chem import AllChem

VINA_CONFIG = """
center_x = {center[0]}
center_y = {center[1]}
center_z = {center[2]}
size_x = {size[0]}
size_y = {size[1]}
size_z = {size[2]}
"""

def opal_run_docking(protein, ligand, center, size, stream_output=True):
    """
    Connect to OPAL webservices and submit job
    """
    client = Client("http://nbcr-222.ucsd.edu/opal2/services/vina_1.1.2?wsdl")
    files = 'receptor.pdbqt', 'ligand.pdbqt', 'vina.conf'
    with open(protein) as f:
        protein_contents = f.read()
    with open(ligand) as f:
        ligand_contents = f.read()
    file_map = [
        {'name': 'receptor.pdbqt',
         'contents': base64ify(protein_contents)},
        {'name': 'ligand.pdbqt',
         'contents': base64ify(ligand_contents)},
        {'name': 'vina.conf',
         'contents': base64ify(VINA_CONFIG.format(center=center, size=size))},
        {'name': 'results.pdbqt',
         'contents': ''},
    ]
    cli_args = "--receptor receptor.pdbqt --ligand ligand.pdbqt --config vina.conf --out results.pdbqt"
    
    response = client.service.launchJob(cli_args, inputFile=file_map)
    job_id = response.jobID
    url = f"http://nbcr-222.ucsd.edu/opal-jobs/{job_id}"
    message = "Waiting for job " + url
    while True:
        r = requests.get(url + "/vina.out")
        try:
            r.raise_for_status()
        except:  # output file might not exist yet during the first checks
            iprint(message)
        else:
            iprint(f"{message}\n{r.text}")
        if client.service.queryStatus(job_id).code == 2:
            time.sleep(10)
            continue
        print('\nFinished!')
        break
        
    output_response = client.service.getOutputs(job_id)
    output_files = {
        'stdout.txt': requests.get(output_response.stdOut).text,
        'stderr.txt': requests.get(output_response.stdErr).text,
    }
    for f in output_response.outputFile:
        if f.name in files:
            continue
        r = requests.get(f.url)
        r.encoding = 'utf-8'
        r.raise_for_status()
        contents = r.text
        output_files[f.name] = contents 
        time.sleep(0.1)
    
    return output_files

######################
#
# Utilities
#
######################

import base64

def base64ify(bytes_or_str):
    """
    Mimic Py2k base64encode behavior
    """
    if isinstance(bytes_or_str, str):
        input_bytes = bytes_or_str.encode('utf8')
    else:
        input_bytes = bytes_or_str

    output_bytes = base64.urlsafe_b64encode(input_bytes)
    return output_bytes.decode('ascii')

def iprint(s):
    """
    We can use this function to print outputs, overwriting previous ones, so it
    looks like it's constantly updating :)
    """
    clear_output(wait=True)
    s = s.replace("\n", "<br />")
    display(HTML(f'<pre>{s}</pre>'))

#### Put it all together

Now that all the needed functions are defined, we can create the pipeline:

In [9]:
def step_03_opal(protein, smiles, pdbcomplex):
    """
    Given a protein structure and a list of smiles strings:
    Steps:
        1. Prepare the protein for AutoDock Vina (locally)
        2. Use DoGSiteScorer to find the most probable binding site
        3. For each ligand, use RDKit to write a 3D PDB file and
           run AutoDockVina on OPAL
    
    The whole thing should take around 5-15 mins
    
    The result is a dictionary with the output file contents. We 
    are mainly interested in result['results.pdbqt']
    """
    prepared_protein = opal_prepare_protein(protein)
    center, radius = dogsite_scorer_guess_binding_site(pdbcomplex)
    size = [radius] * 3  # Vina supports non-cubic boxes, but we will use a cube for simplicity
    for i, smile in enumerate(smiles):
        smiles_to_pdb(smile, f'data/ligand{i}.pdb')
        prepared_ligand = opal_prepare_ligand(f'data/ligand{i}.pdb')
        result = opal_run_docking(prepared_protein, prepared_ligand, center, size)
    return result

Run it! `%time` magic will measure how long it takes for us.

In [10]:
# We will only process the first ligand in `smiles`
%time result = step_03_opal(PROTEIN, smiles[:1], COMPLEX)


Finished!
CPU times: user 2.85 s, sys: 289 ms, total: 3.14 s
Wall time: 5min 25s


#### Understanding the output

`result` is a dictionary with several keys, corresponding to the output files and their text contents. We are mainly interested in:

- `results.pdbqt` contains the docked ligands. It's a modified multi-model PDB file. Since we kept the protein rigid, we just need to open each ligand model together with the original protein structure.
- `vina.out` is the text output you see above. It will provide the table-like information.

Save them to disk so we can retrieve them later.

### Visualize docking results

Once the calculation has run and the files have been downloaded, it's time to visualize them! You will see how to do that in part C.

## Discussion

OPAL offers a Vina installation to perform docking calculations for free, but we have to prepare the input files locally before the submission. Part of that preparation involves defining the search area, normally around a known binding site. Instead of guessing visually, we have used the DoGSiteScorer server present in Proteins.plus to calculate the origin and radius of the most probable binding site. These two services used different communication interfaces.

Proteins.plus uses REST, but does not provide a `swagger.json` definition as KLIFS did, so we had to build our own requests manually. Fortunately, the service is simple enough to *only* need a couple of requests... or so we thought! The current API does not allow for custom PDB uploads, so we had to use a handful of GET and POST requests to pretend we were using a normal browser. To guess the correct requests, the Network tab in Chrome Developer Tools was really handy. If you ever need to reverse engineer a webservice, this is one of the tools you can use!

OPAL is built with the XML-based SOAP standard, a bit more cumbersome than JSON-based REST, but `suds` makes it really easy. Although `suds` will report the available methods, these are not very well documented. Fortunately, there are some code examples scattered in some modules of [UCSF Chimera](http://plato.cgl.ucsf.edu/trac/chimera/browser/trunk/libs/WebServices/opal_client.py). Jobs submitted to OPAL are public once you have the job ID and the files are updated in realtime, so we could build a live preview of the calculation output by querying the server every N seconds and updating the `display()`ed HTML dynamically. This trick can also be used to query a calculation happening in your machine: all you need to do is read the file contents, clear the current output and display the new one as an `HTML()` object (see function `iprint()`).


## Quiz

- How can you tell if the docking has run successfully in the remote server?
- Why do we need to prepare the AutoDock Vina input files locally?
- Advanced: Can you find the correct HTTP requests to use [MCule's docking server](https://mcule.com/apps/1-click-docking/) from the Notebook?